In [1]:
import torch
import torch.nn as nn
import torch.functional as F
import torchvision

In [2]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
vgg19 = torchvision.models.vgg19(pretrained=True).eval().to(device)

In [4]:
for name, layer in vgg19.features.named_children():
    print(f"{name}: {layer}")

0: Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
1: ReLU(inplace=True)
2: Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
3: ReLU(inplace=True)
4: MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
5: Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
6: ReLU(inplace=True)
7: Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
8: ReLU(inplace=True)
9: MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
10: Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
11: ReLU(inplace=True)
12: Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
13: ReLU(inplace=True)
14: Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
15: ReLU(inplace=True)
16: Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
17: ReLU(inplace=True)
18: MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
19: Conv2d(256, 512

In [7]:
class StyleTransferer(nn.Module):
    def __init__(self, feature_extractor):
        super(StyleTransferer, self).__init__()

        self.layers = nn.Sequential()
        conv_layer_idx = 1
        conv_count = 0

        for name, layer in feature_extractor.named_children():
            if isinstance(layer, nn.ReLU):
                layer = nn.ReLU(inplace=False)
            if isinstance(layer, nn.MaxPool2d):
                conv_layer_idx += 1
                conv_count = 0
            if isinstance(layer, nn.Conv2d):
                conv_count += 1
                name = f"conv_{conv_layer_idx}_{conv_count}"

            self.layers.add_module(name, layer)


            

In [10]:
model = StyleTransferer(vgg19.features)
for name, layer in model.layers.named_children():
    print(f"{name}: {layer}")

conv_1_1: Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
1: ReLU()
conv_1_2: Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
3: ReLU()
4: MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
conv_2_1: Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
6: ReLU()
conv_2_2: Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
8: ReLU()
9: MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
conv_3_1: Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
11: ReLU()
conv_3_2: Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
13: ReLU()
conv_3_3: Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
15: ReLU()
conv_3_4: Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
17: ReLU()
18: MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
conv_4_1: Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), p